In [ ]:
import os
from gasp.gt.torst import rsts_to_gpkg

gpkg = {
    "clc18_l2_10x10.gpkg" : '/home/jasp/fireloc/clc_18_l2_rst',
    "clc18_l3_10x10.gpkg" : '/home/jasp/fireloc/clc_18_l3_rst',
    "cos15_l3_10x10.gpkg" : '/home/jasp/fireloc/cos_15_l3_rst',
    "cos15_l5_10x10.gpkg" : '/home/jasp/fireloc/cos_15_l5_rst',
    "cos18_l2_10x10.gpkg" : '/home/jasp/fireloc/cos_18_l2_rst',
    "cos18_l3_10x10.gpkg" : '/home/jasp/fireloc/cos_18_l3_rst',
    "cos18_l4_10x10.gpkg" : '/home/jasp/fireloc/cos_18_l4_rst',
    "forestype15_10x10.gpkg" : '/home/jasp/fireloc/rst_forestype15',
    "imd15_10x10.gpkg" : '/home/jasp/fireloc/rst_imd15',
    "ual2_l2_10x10.gpkg" : '/home/jasp/fireloc/rst_ua12l2',
    "ua12_l3_10x10.gpkg" : '/home/jasp/fireloc/rst_ua12l3',
    "ua12_l4_10x10.gpkg" : '/home/jasp/fireloc/rst_ua12l4',
    "waw15_10x10.gpkg" : "/home/jasp/fireloc/rst_waw15",
    "tct15_10x10.gpkg" : "/home/jasp/fireloc/tcd15_rst"
}

#inrst    = '/home/jasp/mrgis/srtm_bspline_v2'
#out      = '/home/jasp/mrgis/srtm_v2_10x10.gpkg'
new_name = 'cell_' 

for g in gpkg:
    rsts_to_gpkg(gpkg[g], os.path.join(
        os.path.dirname(gpkg[g]), g
    ), rst_ff='.tif', basename=new_name)

### Convert All Rasters in Folder to Shapefile ###

In [ ]:
# Convert using GRASS GIS

import os
from gasp.to.shp  import rst_to_polyg
from gasp.oss     import lst_ff
from gasp.cpu.grs import run_grass
from gasp.oss import fprop

GRASS_WORKSPACE = r'D:\indev\xyz'
GRASS_LOCATION  = 'conv_viena'
RST_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
OUT_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
SRS = 3857
RST_TEMPLATE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena\v14_viena_2x2.tif'

# List Raster Files
rstFiles = lst_ff(RST_FOLDER, file_format='tif')

# Start GRASS GIS Session
grsbase = run_grass(
    GRASS_WORKSPACE, location=GRASS_LOCATION, srs=SRS)

import grass.script.setup as gsetup
gsetup.init(grsbase, GRASS_WORKSPACE, GRASS_LOCATION, 'PERMANENT')

from gasp.to.rst.grs import rst_to_grs
from gasp.cpu.grs.conf import rst_to_region
from gasp.to.shp.grs import grs_to_shp

# Set region
tmpRst = rst_to_grs(RST_TEMPLATE, fprop(RST_TEMPLATE, 'fn'), as_cmd=True)
rst_to_region(tmpRst)

for rst in rstFiles:
    # Add data to GRASS GIS
    grsRst = rst_to_grs(rst, fprop(rst, 'fn'), as_cmd=True)
    
    # Convert to Polygon
    shpGrs = rst_to_polyg(
        grsRst, grsRst + "_shp",
        rstColumn="value", gisApi='grasscmd'
    )
    
    # Export to File
    grs_to_shp(shpGrs, os.path.join(OUT_FOLDER, grsRst + '.shp'), 'area')